In [30]:
import pandas as pd
import wrds

# 1) Connect to WRDS and pull CCMXPF_LNKUSED
db = wrds.Connection()



WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [34]:
lnkused = db.get_table(library='crsp', table='ccmxpf_lnkused')


In [11]:
lnkused.to_csv("output.csv", index=False)

In [32]:
total_gvkeys = lnkused["ugvkey"].nunique()
print(total_gvkeys)

#how many distinct GVKEYs ever appear with a “non‐P” flag.
nonprim_gvkeys = lnkused.loc[lnkused["ulinkprim"].isin(["C","j","N"]), "ugvkey"].unique()

print(len(nonprim_gvkeys))

34575
28323


In [33]:
#Checking wether there might be some Na in the ulinkprim category
print(lnkused["ulinkprim"].isna().sum())

#how many row of each ulinkprim there are:
print(lnkused["ulinkprim"].value_counts())

#P: Primary common share
#C: Co-primary share
#J: Link exist because of complexe corporate event
#N: Often this appears when a firm changes share codes mid‐quarter, 
#   or for a very brief test listing that never became the main security.


#Given the important number of co-primary share it seems irrelevant to drop whatever is not p
#The issue here is that for one gvkey we have different permco
#So when we will agregate compustat and CRPS data we will have different return for same fundamental accounting variable
#By keeping it we will models every investable vehicle (P / C) and study transitional return patterns (J)
#Encode the class code with a categorical variable so that the Deep Learning model can learn new patterns
#We might face some weighting concerns: Firms with multiple share classes apprea multiple times
#Maybe consider by setting a weight = 1/(#of share classes active in that quarter)
len(lnkused)

0
ulinkprim
C    45476
P    45332
N     4001
J     2728
Name: count, dtype: Int64


97537

In [39]:
# 2. Convert/link‐date columns to timestamps, fill NaT end dates
lnkused["ulinkdt"]    = pd.to_datetime(lnkused["ulinkdt"])
lnkused["ulinkenddt"] = pd.to_datetime(lnkused["ulinkenddt"])

#Modifying the Nan (still active key) so that it's easier to filter when we merge the data:
lnkused["ulinkenddt"] = lnkused["ulinkenddt"].fillna(pd.Timestamp("2099-12-31"))

# 3. Keep only the “used” links :
lnkused = lnkused[lnkused["usedflag"] == 1]
#Sometimes a firm’s GVKEY might have a PERMNO assigned for a short “test” period or during a corporate event, 
# but the CRSP/Compustat merge logic did not end up using that PERMNO as part of the official dataset. 
# If you let usedflag = 0 rows through, you might attach a PERMNO that CRSP/Compustat themselves 
# did not consider “valid for analysis.”


lnkused = lnkused[lnkused["ulinkprim"].isin(["P", "C", "J"])]     # (only if you want to force primary‐share)
#magine a company (GVKEY = 1234) that—at the same point in time—has two different share classes trading.
#Eg permno 300001 and permno 30002 that correspon to ordinary and prefered share
#if we do it when we merge crsp and compustat we will have severl time the same compustat information 
#for diferent securities 
#Having the same fundamental values appear twice (or more) 
# in the merged dataset can subtly (or severely) bias almost any empirical test or portfolio‐formation strategy
#TOO SEE IF I REALLY FILTER FOR THIS: If I keep it's better to create a dummy so that the transformer can learn
#something from this and avoid a situation where the same input leads to different results (confusion)

# 4. Select just the columns you need
merging_ready = lnkused[[
    "ugvkey",      # Compustat firm ID
    "upermno",     # CRSP security ID
    "ulinkdt",     # Link start
    "ulinkenddt",  # Link end
    "ulinkprim"    #Kind of securities (P,C,J)
]]

# At this point, lnk_trimmed has exactly the four columns you need. Drop everything else.
merging_ready.head()

,ugvkey,upermno,ulinkdt,ulinkenddt,ulinkprim
2,001000,25881.0,1970-11-13,1978-06-30,P
4,001001,10015.0,1983-09-20,1986-07-31,P
7,001002,10023.0,1972-12-14,1973-06-05,C
10,001003,10031.0,1983-12-07,1989-08-16,C
14,001004,54594.0,1972-04-24,2099-12-31,P


In [40]:
merging_ready.to_csv("merging_keys_ready.csv", index = False)

In [ ]:
#ugvkey: The compustat firm identifier (GVKEY) for this link row
#ulinkprim: 
#uuid : unic IDcode for this particular link
#ulinktype: the link type (LU: used / NU: new but not necessarly used)
#apermno: alias PERMNO ( if a security had more than on this field carry the old one)
#upermno: the CRSP securitiy identiier
#usedflag:  indicating whether this particular PERMNO–GVKEY link was actually used in building the official CRSP‐Compustat combined database.
#ulinkdt: The “link start date” (the first calendar day on which this PERMNO ↔ GVKEY association became valid).
#ulinkenddt: The “link end date” (the last calendar day on which this PERMNO ↔ GVKEY association was valid)